In [2]:
# Chapter 7.1 - 길이 기반 예시 선택자 (LengthBasedExampleSelector)

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# =============================================================================
# 길이 기반 예시 선택자 예제
# =============================================================================
print("=" * 60)
print("길이 기반 예시 선택자 (LengthBasedExampleSelector)")
print("=" * 60)

print("""
원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- chain.run() → chain.invoke()
""")

# 많은 예시 데이터 (원서보다 확장)
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    },
    {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    },
    {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    },
    {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    },
    {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    },
    {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    },
    {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

# 예시 템플릿
example_template = """User: {query}
AI: {answer}"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# 길이 기반 예시 선택자 (핵심 개념!)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # 프롬프트 최대 길이 제한
)

print(f"전체 예시 개수: {len(examples)}")

# 프롬프트 구성
prefix = """The following are excerpts from conversations with an AI assistant. The assistant
is typically sarcastic and witty, producing creative and funny responses to users' questions.
Here are some examples:"""

suffix = """User: {query}
AI:"""

# 동적 프롬프트 템플릿
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # 고정 examples 대신 선택자 사용
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# =============================================================================
# 길이에 따른 예시 선택 테스트
# =============================================================================
print("\n길이에 따른 예시 선택 테스트:")

# 짧은 질문
short_query = "Hi"
formatted_short = dynamic_prompt_template.format(query=short_query)
print(f"\n짧은 질문 '{short_query}'에 대한 프롬프트:")
print(f"선택된 예시 개수: {len(formatted_short.split('User:')) - 2}")

# 긴 질문  
long_query = "Can you explain in detail the comprehensive methodology and step-by-step process for implementing a robust machine learning pipeline?"
formatted_long = dynamic_prompt_template.format(query=long_query)
print(f"\n긴 질문에 대한 프롬프트:")
print(f"선택된 예시 개수: {len(formatted_long.split('User:')) - 2}")

# =============================================================================
# 최신 방식으로 실행
# =============================================================================
print("\n" + "=" * 60)
print("실행 결과")
print("=" * 60)

# 최신 방식: RunnableSequence
chain = dynamic_prompt_template | llm

test_queries = [
    "Who invented the telephone?",
    "What's the meaning of life?",
    "Can you solve this extremely complex mathematical equation involving quantum mechanics and relativity theory?"
]

try:
    for query in test_queries:
        response = chain.invoke({"query": query})
        print(f"\n질문: {query}")
        print(f"답변: {response.content}")
        print("-" * 40)
        
except Exception as e:
    print(f"오류 발생: {e}")

# =============================================================================
# 기존 방식과 비교
# =============================================================================
print("\n" + "=" * 60)
print("기존 방식 vs 새로운 방식")
print("=" * 60)

print("""
원서 코드 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)
response = chain.run({"query": "question"})

수정된 코드 (현재 권장):
chain = dynamic_prompt_template | llm
response = chain.invoke({"query": "question"})

핵심 개념 - LengthBasedExampleSelector:
- 프롬프트 길이를 제한하여 토큰 사용량 최적화
- 입력 쿼리 길이에 따라 적절한 수의 예시 자동 선택
- API 비용 절감과 성능 향상에 효과적
""")

print("\n길이 기반 예시 선택자 예제 완료!")

기존 환경 변수의 API 키를 사용합니다.
길이 기반 예시 선택자 (LengthBasedExampleSelector)

원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- chain.run() → chain.invoke()

전체 예시 개수: 8

길이에 따른 예시 선택 테스트:

짧은 질문 'Hi'에 대한 프롬프트:
선택된 예시 개수: 5

긴 질문에 대한 프롬프트:
선택된 예시 개수: 4

실행 결과

질문: Who invented the telephone?
답변: Alexander Graham Bell, the man who made it possible for us to have awkward phone conversations with our relatives.
----------------------------------------

질문: What's the meaning of life?
답변: To find the perfect balance between eating pizza and going to the gym.
----------------------------------------

질문: Can you solve this extremely complex mathematical equation involving quantum mechanics and relativity theory?
답변: Sure, let me just grab my quantum calculator and my theory of relativity handbook. Oh wait, I'm just a sarcastic AI, not a supercomputer!
----------------------------------------

기존 방식 vs 새로운 방식

원서 코드 (Deprecat